In [22]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import vincent
import warnings
import json
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [10]:
bairro = gpd.read_file('./bairro/bairro-cg.shp')
locais_cg = pd.read_csv('./locais-cg.csv')

In [3]:
with open('./bairro_json/geo_json.json', 'r') as file:
    geo_json = file.read()

In [28]:
y = bairro[bairro['NM_BAIRRO']=='Centro'].centroid.y.iloc[0]
x = bairro[bairro['NM_BAIRRO']=='Centro'].centroid.x.iloc[0]

### Plotando o mapa de campo grande ms

In [5]:
base = folium.Map([y, x], zoom_start=12, tiles='OpenStreetMap')

for i in range(len(bairro)):
    geo = folium.GeoJson(bairro[i:i+1], name=bairro['NM_BAIRRO'][i])
    label = f"{bairro['NM_BAIRRO'][i]} - {bairro['V002'][i]} Habitantes"
    folium.Popup(label).add_to(geo)
    geo.add_to(base)
folium.LayerControl().add_to(base)

base

### colorindo os bairros da cidade de acordo com a população

#### Os dados sobre a população foram retirados do site do IBGE (obs: os dados são de 2010)

In [35]:
base = folium.Map([y, x], zoom_start=12, tiles='OpenStreetMap')

folium.Choropleth(
    geo_data=geo_json,
    data=bairro,
    columns=['NM_BAIRRO', 'V002'],
    key_on='feature.id',
    fill_color='YlOrBr',
    fill_opacity=0.5,
    line_opacity=0.2,
    highlight=True
).add_to(base)
base

In [12]:
locais_cg.head()

,NOME,LONGITUDE,LATITUDE,icons,colors
0,SHOPPING NORTE SUL PLAZA,-54.627472,-20.479197,fa-shopping-cart,red
1,SHOPPING CAMPO GRANDE,-54.588025,-20.458020,fa-shopping-cart,red
2,SHOPPING BOSQUE DOS IPES,-54.560302,-20.396380,fa-shopping-cart,red
3,AEROPORTO INTERNACIONAL,-54.667713,-20.456493,fa-plane,blue
4,BIOPARQUE PANTANAL,-54.575828,-20.456911,fa-star,blue


### Adicionando marcadores ao Mapa

In [20]:
html = """
    <h3>{}</h3>
"""


base = folium.Map([y, x], zoom_start=12, tiles='OpenStreetMap')
folium.GeoJson(geo_json, zoom_on_click=True).add_to(base)

for item in locais_cg.itertuples():
    folium.Marker([
        item.LATITUDE,
        item.LONGITUDE],
        popup=html.format(item.NOME),
        icon=folium.Icon(color=item.colors, prefix='fa',
                         icon=f"fas {item.icons}")
).add_to(base)
base

### Criando Popups com gráficos usando o Vincent

In [34]:
import vincent

b0 = 4.3
b1 = 0.6

xs = np.linspace(0, 200)
ys = [(b1*x)+b0 for x in xs]

line = {
    "x": xs,
    "y": ys
}

line_vincent = vincent.Line(line, iter_idx="x", width=300, height=150)
line_vincent.axis_titles(x='Meses', y='Faturamento')

base = folium.Map([y, x], zoom_start=12, tiles='OpenStreetMap')
folium.GeoJson(geo_json).add_to(base)
popup = folium.Popup()
folium.Vega(line_vincent,  width=370, height=200).add_to(popup)
folium.Marker([-20.453112,-54.601362], popup=popup,
              icon=folium.Icon(color='red', prefix='fa',
                         icon=f"fas fa-shopping-basket")).add_to(base)
base

In [30]:
condominios = pd.read_csv('./condominios.csv')

### Criando Clusters de marcadores

In [31]:
from folium.plugins import MarkerCluster

b0 = 4.3
b1 = 0.6

xs = np.linspace(0, 200)
ys = [(b1*x)+b0 for x in xs]

line = {
    "x": xs,
    "y": ys
}

line_vincent = vincent.Line(line, iter_idx="x", width=300, height=150)
line_vincent.axis_titles(x='Meses', y='Faturamento')

cg = folium.Map([y, x], zoom_start=12, tiles='OpenStreetMap')

marker_cluster = MarkerCluster().add_to(cg)

for item in condominios.itertuples():
    folium.Marker(
        location=[item.LATITUDE, item.LONGITUDE],
        popup=item.NOME,
        icon=folium.Icon(color='orange', prefix='fa', icon='fas fa-building-o')
).add_to(marker_cluster)

    
for item in locais_cg.itertuples():
    if item.NOME == 'SAMS CLUB':
        popup = folium.Popup()
        folium.Vega(line_vincent,  width=370, height=200).add_to(popup)
        folium.Marker([-20.453112,-54.601362], popup=popup,
                      icon=folium.Icon(color='red', prefix='fa',icon=f"fas fa-shopping-basket")).add_to(cg)
    else:
        folium.Marker([
            item.LATITUDE,
            item.LONGITUDE],
            popup=html.format(item.NOME),
            icon=folium.Icon(color=item.colors, prefix='fa',
                             icon=f"fas {item.icons}")
).add_to(cg)
    
cg

### Adicionando o plugin de Busca

In [33]:
from folium.plugins import Search

base = folium.Map([y, x], zoom_start=12, tiles='OpenStreetMap')

strat_geo = folium.GeoJson(
    bairro,
    name='Bairros CG',
    tooltip=folium.GeoJsonTooltip(
        fields=['NM_BAIRRO','V002'], aliases=['bairro','População'])
).add_to(base)

statesearch = Search(
    layer=strat_geo,
    geom_type='Polygon',
    placeholder='Buscar Bairro',
    search_label='NM_BAIRRO',
    weight=3,
    collapsed=True
).add_to(base)
base